In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm  # Progress bar for processing images

# Define the paths
train_male_dir = 'E:/p2/g2/archive/Training/male'
train_female_dir = 'E:/p2/g2/archive/Training/female'
val_male_dir = 'E:/p2/g2/archive/Validation/male'
val_female_dir = 'E:/p2/g2/archive/Validation/female'

# Check if the directories exist
for path in [train_male_dir, train_female_dir, val_male_dir, val_female_dir]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"The specified directory {path} does not exist.")
    print(f"Found directory: {path}")

# Load dataset
data = []
labels = []

# Function to process images in a directory
def process_images_from_directory(directory, label):
    images = os.listdir(directory)
    print(f"Processing {len(images)} images from {directory}")
    for img in tqdm(images, desc=f"Processing {label} images", unit="image"):
        img_path = os.path.join(directory, img)
        image = cv2.imread(img_path)
        if image is not None:
            image = cv2.resize(image, (64, 64))  # Resize image to 64x64 pixels
            data.append(image)
            labels.append(label)
        else:
            print(f"Failed to load image: {img_path}")

# Process Training data
print("Processing Training Data")
process_images_from_directory(train_male_dir, 'male')
process_images_from_directory(train_female_dir, 'female')

# Process Validation data
print("Processing Validation Data")
process_images_from_directory(val_male_dir, 'male')
process_images_from_directory(val_female_dir, 'female')

# Display some sample images
def display_samples(data, labels, num_samples=5):
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
    for i in range(num_samples):
        axes[i].imshow(data[i])
        axes[i].set_title(labels[i])
        axes[i].axis('off')
    plt.show()

# Convert to numpy arrays
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# Encode labels
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)

# Split the data (using only training data for training and validation purposes)
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with progress tracking
print("Training the model")
history = model.fit(trainX, trainY, epochs=2, validation_data=(testX, testY), verbose=1) #20

# Save the model
print("Saving the model")
model.save("gender_detection_model.h5")

# Plot training history
def plot_history(history):
    plt.figure(figsize=(12, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training   Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss over Epochs')
    plt.legend()
    
    plt.show()

# Display sample images from the dataset
print("Displaying sample images from the dataset")
display_samples(data, np.array([le.inverse_transform([label.argmax()])[0] for label in labels]))

# Plot training curves
print("Plotting training history")
plot_history(history)


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the trained model
model = load_model('gender_detection_model.h5')  # or 'gender_detection_transfer.h5' for the transfer learning model
print("Model loaded.")

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
print("Haar Cascade loaded.")

# Define image size
img_size = (64, 64)

# Define labels
labels = {0: 'Female', 1: 'Male'}

# Initialize webcam
cap = cv2.VideoCapture(0)
print("Webcam initialized.")

# Face ID counter
face_id = 0

def preprocess_face(face):
    # Convert to RGB
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    # Resize image
    face = cv2.resize(face, img_size)
    # Convert to array
    face = img_to_array(face)
    # Scale pixel values
    face = face / 255.0
    # Expand dimensions to match input shape of model
    face = np.expand_dims(face, axis=0)
    return face

print("Starting real-time gender detection with face detection...")
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Apply mirror mode (horizontal flip)
    frame = cv2.flip(frame, 1)
    
    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        # Extract the face from the frame
        face = frame[y:y+h, x:x+w]
        
        # Preprocess the face
        processed_face = preprocess_face(face)
        
        # Predict gender
        prediction = model.predict(processed_face)
        gender = labels[np.argmax(prediction)]
        confidence = np.max(prediction)
        
        # Assign a unique ID to each detected face
        face_id += 1
        
        # Print face ID, gender, and confidence to the console
        print(f"Face ID: {face_id}, Gender: {gender}, Confidence: {confidence:.2f}")
        
        # Display the results on the frame
        label = f"ID: {face_id} {gender}: {confidence:.2f}"
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Show the frame
    cv2.imshow('Gender Detection', frame)
    
    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
print("Real-time gender detection ended.")
